File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


In [76]:
month_year = REPORT_DATE.strftime("%m.%Y")
isin_file = f"ISIN_{month_year}.xlsx"
Prices = pd.read_excel(f"./data/Input/{isin_file}", names=["ISIN"]).dropna().set_index("ISIN")
Prices["Date"] = REPORT_DATE
Prices[["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price", 
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]] = np.nan

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [4]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (154712, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,199161,None,None,"Cohen & Steers Total Return Realty Fund Inc, O...",None,US19247R1032,Cohen & Stee-US19247R1032,NaN,None,None,None,NaN,None,None,CICJLS,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,US,NaN,USD,0.001,NaN,NaN,NaN,В обращении,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,Cohen & Steers Total,None,None,119903.0,None,US,None,None,NaN,None,None,NaN,None,None,0.0,None,Обыкн-ая,Неопределен,None,2023-10-12T00:25:07,NaN,Нет информации,NaN,None,NaN,None,NaN,None,None,None,NaN,NaN,None,None,NaN,None,False,None,US,1000,1000,NaN,None,NaN,False,False,None,NaN,107.0,None,None,103712,81114
1,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None

In [5]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (103712, 14)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,124135,0,Calendar,None,2025-04-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,USD,USD,2025-04-30T00:00:00
1,19224,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '19224': Не...,None,None,None
2,168562,0,Calendar,None,2025-04-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-04-30T00:00:00
3,178136,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '178136': Н...,None,None,None
4,7711,0,Calendar,None,2025-04-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-04-30T00:00:00


In [6]:
RUPriceHistory = RuDataDF("RUPriceHistory").df
RUPriceHistory.shape

RUPriceHistory started
RUPriceHistory finished. RUPriceHistory shape (44905, 82)


(44905, 82)

In [7]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (241, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,75,68,ECB,ECB,ECB Eonia,ECB Eonia,76,2020-12-28T17:03:09,241,97
1,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,241,1
2,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,241,2
3,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,241,3
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,241,4


In [8]:
MoexSecurities = RuDataDF("MoexSecurities").df
MoexSecurities.head()

MoexSecurities started
MoexSecurities finished. MoexSecurities shape (214908, 24)


,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn
0,3012222,RUMBCPBBBNS,RTSI,Индексы РТС,stock_index,Индексы,index,stock,RUMBCPBBBNS-МБ,None,ISEXXX,Индексы / акции,1,None,RUB,None,None,None,NaN,173577.0,None,2023-01-24T19:03:21.343901,214908,80499
1,3012223,RUMBCPANS,RTSI,Индексы РТС,stock_index,Индексы,index,stock,МБМуницОблигRUMBCPANS-МБ,None,ISEXXX,Индексы / акции,1,None,RUB,None,None,None,NaN,173578.0,None,2023-01-24T19:02:27.343899,214908,80500
2,3012224,RUMBCPAANS,RTSI,Индексы РТС,stock_index,Индексы,index,stock,МБМуницОблигRUMBCPAANS-МБ,None,ISEXXX,Индексы / акции,1,None,RUB,None,None,None,NaN,173579.0,None,2023-01-24T19:01:42.343897,214908,80501
3,3012225,RUMBCPAAANS,RTSI,Индексы РТС,stock_index,Индексы,index,stock,RUMBCPAAANS-МБ,None,ISEXXX,Индексы / акции,1,None,RUB,None,None,None,NaN,173580.0,None,2023-01-24T19:00:14.343895,214908,80502
4,3012226,RUMBCP3A3YNS,RTSI,Индексы РТС,stock_index,Индексы,index,stock,RUMBCP3A3YNS,None,ISEXXX,Индексы / акции,1,None,RUB,None,None,None,NaN,173588.0,None,2023-01-24T18:44:17.343879,214908,80503


In [9]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI

isins = Prices[Prices["Price"].isna()].index.unique().tolist()
key = "EndOfDay"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
payloads = []
for isin in isins:
    payload["isin"] = isin
    payloads.append(payload.copy())
    
async def end_of_day_rudata():
    end_of_day = []
    async with aiohttp.ClientSession(
            connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
                                           family=socket.AF_INET),
            trust_env=True,
            timeout=aiohttp.ClientTimeout(3600)
    ) as session:
        for i, chunk in enumerate(divide_chunks(payloads, DocsAPI.LIMIT)):
            tasks: List[asyncio.Task] = [asyncio.create_task(
                RuDataRequest(url, session).post(payload=payload)
            ) for payload in chunk]
            resAll: List[List[dict]] = [await task for task in asyncio.as_completed(tasks, timeout=600)]
            result: List[dict] = [row for task_res in resAll for row in task_res]
            end_of_day.extend(result)
    return end_of_day
    
end_of_day = asyncio.run(end_of_day_rudata())
EndOfDay = pd.DataFrame(end_of_day)
# EndOfDay: pd.DataFrame = pd.read_sql(
#                     f"""
#                     SELECT *
#                     FROM "EndOfDay"
#                     where report_monthyear = '{RuDataDF.report_monthyear}'
#                     """
#                     , RuDataDF.engine
#                 )
EndOfDay.head()

CPU times: user 9.11 s, sys: 2.07 s, total: 11.2 s
Wall time: 28min 31s


,accruedint,admquote,akm_bl_n,akm_e_n,ask,ask_size,ask_size_total,avge_chng,avge_chng_pct,avge_prce,avge_yield,bid,bid_size,bid_size_total,boardid,boardname,buybackdate,buybackprice,cbr_lombard,cbr_pledge,chng,chng_m,chng_m_pct,chng_pct,chng_w,chng_w_pct,chng_y,chng_y_pct,close,consensus_num_est,consensus_time,convexity,convexity_agg,convexity_o,cpn,cpn_date,cpn_period,cpprice,deal_acc,discount,dnt06beg,dnt06max,dnt06min,dnt120beg,dnt120max,dnt120min,dnt14beg,dnt14max,dnt14min,dnt200beg,dnt200max,dnt200min,dnt365beg,dnt365max,dnt365min,duration,duration_agg,duration_n,duration_o,duration_on,exch,exp_bl_n,exp_e_n,facevalue,facevalue_currency,fact_frcst_chng,fch_b_i,fch_b_n,fch_e_dlti,fch_e_lti,forecast,frcst_chng,high,ifx_b_i,ifx_b_n,ifx_e_i,ifx_e_n,isin,issuer,issuesize,last,last_time,last_yield,last2avge,lastsettlecode,lclose,lnkcurrency,lotsize,low,mat_date,mat_date_dt,mcap,mds_b_i,mds_b_n,mds_e_lti,mds_e_ltn,mprice,mprice2,name,open,...,pvbp_o,rec_buy_pct,rec_change,rec_hold_pct,rec_sell_pct,rec_val,recommendation,rus_b_n,rus_e_i,rus_e_n,seccode,secname,secname_e,sector,settlecode,shortname_eng,shortname_rus,snp_b_i,snp_b_n,snp_e_lti,snp_e_n,spread,status,target_price,tick_acc,time,tp_chng,tp_chng_m,tp_chng_m_pct,tp_chng_pct,tp_chng_q,tp_chng_q_pct,tp_chng_time,tp_chng_y,tp_chng_y_pct,tp_deviation,tp_high,tp_low,tp_median,val,val_acc,val_acc_usd,val_usd,vol,vol_acc,vol_lots,y2o_ask,y2o_bid,y2o_change,y2o_change_pct,y2o_last,y2o_prev,y2o_wap,yield_2,yield_agg,yield_ask,yield_average_12m,yield_average_1m,yield_average_2m,yield_average_2w,yield_average_3m,yield_average_6m,yield_average_o12m,yield_average_o1m,yield_average_o2m,yield_average_o2w,yield_average_o3m,yield_average_o6m,yield_bid,yield_chng,yield_cpprice,yield_cwa,yield_high,yield_low,yield_o_cwa,yield_o_swa,yield_open,yield_prev,yield_swa,total_return,full_price_pct,zspread,gspread,currency,fintoolId,id_iss,lcurrentprice,prevlclose,lcp_chng,lcp_chng_pct,bid_price_change,ask_price_change,bid_yield_change,ask_yield_change,id_trade_site,ifrs_level,rd_exch,rd_score,add_date,update_date
0,39.810000,None,None,None,100.30,None,19805.0,0.40,0.40,100.25,18.648956,99.75,None,1353.0,TQCB,Т+: Корп.облигации - безадрес.,2028-11-02T03:00:00,100.0,99.65,None,0.31,0.98,0.98,0.31,0.50,0.50,5.98,6.36,99.98,None,None,33.319998,0.027162,0.027162,41.88,2025-05-08T00:00:00,182.0,None,5.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1869.0,9.0,4.316991,9.0,0.025566,MICEX,None,None,1000.0,None,None,None,None,None,None,None,None,100.30,None,None,None,None,RU000A0JXR84,None,15000000.0,100.29,2025-04-29T15:11:04,18.637845,0.44,None,None,RUB,1.0,99.75,23.04.2037,2037-04-23T00:00:00,1.503750e+09,None,None,None,None,99.8500,None,RZHD01P3R.MICEX,100.30,...,-0.002666,None,None,None,None,None,None,None,None,None,RU000A0JXR84,"""Российские ЖД"" БО-001P-03R",RZhD BO-001P-03R,None,None,None,РЖД-001П-03БР,None,None,None,None,0.55,None,NaN,None,2025-04-29T00:00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,51125.0,51125.0,619.32,619.32,51.0,51.0,51.0,-3.55376,18.585205,NaN,NaN,-3.55376,NaN,-1.659522,None,-3.553760,18.637845,9.261543,9.081002,8.865733,9.535196,8.969374,9.119492,20.533581,18.180456,18.692413,17.128562,20.882539,22.374857,18.756113,NaN,None,8.379052,18.760595,18.637845,8.379052,-1.6731,18.637845,NaN,17.181724,None,104.271,None,None,RUB,132511,1407621,None,None,None,None,None,None,None,None,170,NaN,None,None,2025-04-29T23:55:33,2025-04-29T23:55:33
1,37.750000,None,None,None,95.76,None,34383.0,-0.08,-0.08,95.62,27.010879,95.60,None,2194.0,TQCB,Т+: Корп.облигации - безадрес.,None,NaN,NaN,None,0.00,-0.66,-0.68,0.00,-0.30,-0.31,-4.58,-4.57,95.60,None,None,3.490262,3.490262,NaN,55.17,2025-05-29T00:00:00,91.0,None,23.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,657.0,657.0,1.417249,NaN,NaN,MICEX,None,None,1000.0,None,None,None,None,None,None,None,None,95.70,None,None,None,None,RU000A106P97,None,30000000.0,9

In [29]:
EndOfDay['report_monthyear'] = last_day_month.strftime("%m%Y")

In [30]:
EndOfDay.to_sql(
                name='EndOfDay',
                con=RuDataDF.engine,
                if_exists='append',
                index=False)

99

In [11]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep

currencies = (
    pd.read_sql(
        """
        SELECT DISTINCT currency
        FROM currencies
        """
        , RuDataDF.engine
    )['currency']
    .to_list()
)
key = "CurrencyRate"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
for currency in currencies:
    payload["from"] = currency
    ans = requests.post(url, json={'from': currency, 'to': 'RUB', 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    result: dict = ans.json()
    result["currency"] = currency
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
# CurrencyRate: pd.DataFrame = pd.read_sql(
#                     f"""
#                     SELECT *
#                     FROM "CurrencyRate"
#                     where report_monthyear = '{RuDataDF.report_monthyear}'
#                     """
#                     , RuDataDF.engine
#                 )
CurrencyRate.head()

CPU times: user 3.91 s, sys: 437 ms, total: 4.35 s
Wall time: 5min 57s


,rate,endOfDayRate,error,currency
0,19.842100,19.842100,None,LTL
1,2.447170,2.439990,None,THB
2,NaN,NaN,None,CDF
3,10.530900,10.524800,None,HKD
4,0.004837,0.004855,None,IDR


In [31]:
CurrencyRate['report_monthyear'] = last_day_month.strftime("%m%Y")

In [32]:
CurrencyRate.to_sql(
                name='CurrencyRate',
                con=RuDataDF.engine,
                if_exists='append',
                index=False)

166

##### Шаг 1.1. Discounts

###### Шаг 1.1.1. CBR

In [77]:
from sources.CBR_Soap import CBR_Soap

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.6834.111 Safari/537.36', 
           'Accept-Encoding': 'gzip, deflate', 
           'Accept': '*/*', 
           'Connection': 'keep-alive'}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.set_index('ISIN')

GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.10


/Users/alexander/PycharmProjects/insurance_mine/sources/CBR_Soap.py:212: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  disc = pd.concat([disc, pd.DataFrame(index=[0], data=d)], axis=0)


In [78]:
Prices.update(cbr_disc)
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [79]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecurities[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()
currMarketRates = currMarketRates[currMarketRates.index.str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)]
currMarketRates=currMarketRates[['Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))) #Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates /= 100

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/88371159.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)


In [80]:
Prices.update(currMarketRates)
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [81]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.set_index('ISIN')
securInfo = securInfo[securInfo['NCC collateral']=='Да']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/1680612647.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\


In [82]:
Prices.update(securInfo)
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [83]:
TransformFintoolReferenceData = FintoolReferenceData[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type", 
    "faceftname": "Currency"
})
TransformFintoolReferenceData["ISIN in DB"] = 1
TransformFintoolReferenceData["Source"] = 'RU Data (ЦЦ НРД)'
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    AccruedInterestOnDate[['fintoolId', 'accruedInterest', 'currentFaceValue']], how="left", left_on="Fintool", right_on="fintoolId")
TransformFintoolReferenceData = TransformFintoolReferenceData\
.rename(columns={
    'accruedInterest': 'Interest', 
    'currentFaceValue': 'Principal'})\
.drop(columns=["fintoolId"])

In [84]:
RUPriceHistoryLast = RUPriceHistory.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()

In [85]:
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
TransformFintoolReferenceData["Interest"] = TransformFintoolReferenceData["Interest"].combine_first(TransformFintoolReferenceData["InterestHistory"])
TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)
TransformFintoolReferenceData.drop(columns="InterestHistory", inplace=True)

In [86]:
bonds_index = TransformFintoolReferenceData[TransformFintoolReferenceData["Type"]=='Облигация'].index

In [87]:
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"] + \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Interest"]

TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"]

In [88]:
Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())

In [89]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [90]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/1720937637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/1720937637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/1720937637.py:4: SettingWithCopyWarning: 
A value

In [91]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/3772375966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/3772375966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Price'] = exch_not_bonds['PriceEX']


In [94]:
Prices.update(exch_bonds.drop_duplicates().set_index("ISIN"))
Prices.update(exch_not_bonds.drop_duplicates().set_index("ISIN"))
# Prices.update(exch_not_bonds.set_index("ISIN"))

In [95]:
ResultPrices = Prices[~Prices['Price'].isna()]
Prices = Prices[Prices['Price'].isna()]

#### Шаг 2. Cbonds

##### Шаг 2.1. Bonds

In [96]:
tradings_bonds = pd.read_excel(f'./data/Input/tradings_bonds.xls')
tradings_bonds_1 = pd.read_excel(f'./data/Input/tradings_bonds_1.xls')

tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)
tradings_bonds = tradings_bonds[['ISIN', 'Краткое название эмитента (rus)', 'Валюта', 'Индикативная цена', 'Номинал по еврооблигациям']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта': 'Currency', 
    'Индикативная цена': 'Clean price pct', 
    'Номинал по еврооблигациям': 'Principal EB'
})

In [97]:
tradings_bonds = tradings_bonds.merge(Prices[["Interest", "Principal"]], how='inner', left_on="ISIN", right_index=True)
principal_idx = tradings_bonds[tradings_bonds['Principal']>tradings_bonds['Principal EB']].index
tradings_bonds.loc[principal_idx, 'Principal'] /= tradings_bonds.loc[principal_idx, 'Principal EB']
tradings_bonds.loc[principal_idx, 'Interest'] /= tradings_bonds.loc[principal_idx, 'Principal']

In [98]:
Prices.update(tradings_bonds.set_index("ISIN"))

In [99]:
tradings_nkd = pd.read_excel(f'./data/Input/tradings_nkd.xls')
tradings_nkd_1 = pd.read_excel(f'./data/Input/tradings_nkd_1.xls')
tradings_nkd_2 = pd.read_excel(f'./data/Input/tradings_nkd_2.xls')
tradings_nkd_3 = pd.read_excel(f'./data/Input/tradings_nkd_3.xls')

tradings_nkd = pd.concat([tradings_nkd, tradings_nkd_1, tradings_nkd_2, tradings_nkd_3], axis=0)
tradings_nkd = tradings_nkd[['ISIN код эмиссии', 'Накопленный купонный доход', 'Непогашенный номинал']]\
.rename(columns={
    'ISIN код эмиссии': "ISIN", 
    'Накопленный купонный доход': "Interest", 
    'Непогашенный номинал': "Principal"
})
tradings_nkd = tradings_nkd.groupby("ISIN").first()

In [100]:
Prices.update(tradings_nkd)

In [101]:
tradings_new = pd.read_excel(f'./data/Input/tradings_new_{REPORT_DATE_STR}.xls')

tradings_new = tradings_new[['Код ISIN', 'Эмиссия', 'Дата', 'Bid, закр.', 'НКД']]\
.rename(columns={
    'Код ISIN': 'ISIN',
    'Эмиссия': 'Name',
    'Дата': 'Date',
    'Bid, закр.': 'Clean price pct',
    'НКД': 'Interest'
})
tradings_new["Max date"] = tradings_new.groupby('ISIN')["Date"].transform('max')
tradings_new = tradings_new.groupby('ISIN').first()

In [102]:
emissions = pd.read_excel(f'./data/Input/emissions_{REPORT_DATE_STR}.xls')

emissions = emissions[['Валюта', 'код ISIN', 'Непогашенный номинал']]\
.rename(columns={
    'код ISIN': 'ISIN',
    'Валюта': 'Currency',
    'Непогашенный номинал': 'Principal'
})
emissions = emissions[~((emissions["ISIN"].isna()) | (emissions["Principal"].isna()) | (emissions["Principal"]==0))].set_index("ISIN")
tradings_new = tradings_new.merge(emissions, how="left", left_index=True, right_index=True)

In [103]:
Prices.update(tradings_new)

In [104]:
Prices.loc[Prices['Interest'] > 0.2*Prices['Principal'], 'Interest'] = 0
Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
Prices['Price'] = Prices['Clean price'] + Prices['Interest']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/2901201166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_17693/2901201166.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Prices['Price'] = Prices['Clean price'] + Prices['Interest']


In [105]:
cbonds_bonds_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[cbonds_bonds_idx, "Type"] = 'Облигация'
Prices.loc[cbonds_bonds_idx, 'Source'] = 'Cbonds'
Prices.loc[cbonds_bonds_idx, 'ISIN in DB'] = 1

In [106]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[cbonds_bonds_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

##### Шаг 2.2. Stocks

In [107]:
tradings_stocks_all = pd.read_excel(f'./data/Input/tradings_stocks_all.xls')

tradings_stocks_all = tradings_stocks_all[['ISIN', 'Краткое название эмитента (rus)', 'Валюта номинала', 'Валюта торгов', 'Цена закрытия']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта номинала': 'Currency', 
    'Валюта торгов': 'Trade currency', 
    'Цена закрытия': 'Price'
})

In [108]:
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Trade currency", right_on="currency")\
.rename(columns={'rate': 'trade rate'})
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Currency", right_on="currency")
tradings_stocks_all['RUB Price'] = tradings_stocks_all['Price'] * tradings_stocks_all['trade rate']
tradings_stocks_all['RUB Price'] = tradings_stocks_all.groupby("ISIN")['RUB Price'].transform('mean')
tradings_stocks_all['Currency'] = tradings_stocks_all['Currency'].combine_first(tradings_stocks_all['Trade currency'])
tradings_stocks_all['Clean price'] = tradings_stocks_all['RUB Price'] / tradings_stocks_all['rate']
tradings_stocks_all['Price'] = tradings_stocks_all['Clean price']
tradings_stocks_all['FX Rate'] = tradings_stocks_all['trade rate'].combine_first(tradings_stocks_all['rate'])

In [109]:
cbonds_stocks_idx = tradings_stocks_all[~tradings_stocks_all["Price"].isna()].index
tradings_stocks_all.loc[cbonds_stocks_idx, "Type"] = 'Акция'
tradings_stocks_all.loc[cbonds_stocks_idx, 'Source'] = 'Cbonds'
tradings_stocks_all.loc[cbonds_stocks_idx, 'ISIN in DB'] = 1

In [110]:
Prices.update(tradings_stocks_all.groupby("ISIN").first())

In [111]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices["Price"].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [112]:
Prices.shape

(1750, 16)

#### Шаг 3. MOEX

##### Шаг 3.0. RuData MOEX stage

In [113]:
HistoryStockBonds = RuDataDF("HistoryStockBonds").df
HistoryStockBonds.head()

HistoryStockBonds started
HistoryStockBonds finished. HistoryStockBonds shape (145549, 66)


,boardname,regnumber,isin,matdate,listname,facevalue,currencyid,volume,value,numtrades,prev,prevlegalcloseprice,openperiod,legalopenprice,legalcloseprice,closeperiod,waprice,trendclose,trendclspr,trendwap,trendwappr,openval,closeval,highbid,lowoffer,bid,offer,yieldatwap,yieldclose,accint,duration,marketprice,marketprice2,admittedquote,issuesize,mpvaltrd,mp2valtrd,admittedvalue,marketprice3,marketprice3tradesvalue,type,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Облигации - безадрес.,4-01-36400-R-003P,RU000A105A95,2034-04-28T00:00:00,3,1000.0,SUR,592,52979731.89,214,104.6551,104.8000,105.2894,None,105.2862,None,105.2659,0.1470,0.14,0.5075,0.48,89512.73,89098.45,105.8273,104.5044,104.8044,105.2047,7.84,7.91,3319.8709,2298.0,None,105.2678,None,748626,0.0,51727761.81,None,105.2678,51727761.81,corporate_bond,104.8021,NaN,NaN,8.63,43.13,None,2025-04-11T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,105.8273,104.4972,105.2894,104.8021,RU000A105A95,ГазКЗ-34Д,2025-04-11T00:00:00,3,NaN
1,Т+: Облигации - безадрес.,4-01-36400-R-003P,RU000A105A95,2034-04-28T00:00:00,3,1000.0,SUR,372,32751256.16,193,104.8021,105.2862,NaN,None,104.5913,None,104.8057,-0.3782,-0.36,-0.4602,-0.44,88037.96,87720.25,105.9799,104.3403,104.3051,104.5500,7.91,7.97,3340.8391,2295.0,None,104.8597,None,748626,0.0,27923374.93,None,104.8597,27923374.93,corporate_bond,104.4239,NaN,NaN,8.63,43.13,None,2025-04-14T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,105.9799,104.3000,104.8021,104.4239,RU000A105A95,ГазКЗ-34Д,2025-04-14T00:00:00,3,NaN
2,Т+: Облигации - безадрес.,4-01-36400-R-003P,RU000A105A95,2034-04-28T00:00:00,3,1000.0,SUR,691,59962346.63,233,104.4239,104.5913,NaN,None,105.2122,None,104.8438,1.7861,1.71,0.0381,0.04,86492.61,87906.94,106.2104,103.4679,106.2100,106.2300,7.90,7.69,3311.5117,2294.0,None,104.8071,None,748626,0.0,58119682.08,None,104.8071,58119682.08,corporate_bond,106.2100,NaN,NaN,8.63,43.13,None,2025-04-15T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,106.3727,104.2804,104.5012,106.2100,RU000A105A95,ГазКЗ-34Д,2025-04-15T00:00:00,3,NaN
3,Т+: Облигации - безадрес.,4-01-36400-R-003P,RU000A105A95,2034-04-28T00:00:00,3,1000.0,SUR,549,48702270.42,177,106.2100,105.2122,106.9000,None,107.9500,None,107.7892,1.6690,1.57,2.9454,2.81,351916.08,887847.41,109.3556,106.4996,107.8791,108.2898,7.45,7.44,3312.5871,2309.0,None,107.7764,None,748626,0.0,45237139.01,None,107.7764,45237139.01,corporate_bond,107.8790,NaN,NaN,8.63,43.13,None,2025-04-16T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,108.2951,106.3010,106.9000,107.8790,RU000A105A95,ГазКЗ-34Д,2025-04-16T00:00:00,3,NaN
4,Т+: Облигации - безадрес.,4-01-36400-R-003P,RU000A105A95,2034-04-28T00:00:00,3,1000.0,SUR,491,43181834.32,196,107.8790,107.9500,NaN,None,106.5996,None,106.4869,-1.8281,-1.69,-1.3023,-1.21,89194.93,87586.59,108.0044,105.0000,106.0505,106.5818,7.65,7.72,3344.0367,2301.0,None,106.5853,None,748626,0.0,28344990.76,None,106.5853,28344990.76,corporate_bond,106.0509,NaN,NaN,8.63,43.13,None,2025-04-17T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,107.9984,106.0509,107.9983,106.0509,RU000A105A95,ГазКЗ-34Д,2025-04-17T00:00:00,3,NaN


In [114]:
HistoryStockShares = RuDataDF("HistoryStockShares").df
HistoryStockShares.head()

HistoryStockShares started
HistoryStockShares finished. HistoryStockShares shape (32137, 54)


,boardname,regnumber,isin,listname,facevalue,currencyid,prevlegalcloseprice,prev,legalopenprice,openperiod,closeperiod,openval,closeval,legalcloseprice,trendclose,trendclspr,highbid,bid,offer,lowoffer,waprice,trendwap,trendwappr,volume,marketprice,marketprice2,mp2valtrd,mpvaltrd,value,numtrades,issuesize,admittedquote,admittedvalue,monthlycapitalization,dailycapitalization,marketprice3,marketprice3tradesvalue,type,closeauctionprice,waval,lastprice,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Акции и ДР - безадрес.,1-01-04247-E,RU000A0JPPN4,2,0.1,SUR,NaN,0.06875,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,188307958732,None,None,1.344519e+10,1.288026e+10,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,TQBR,5,NaN,NaN,NaN,NaN,MRKV,РсетВол ао,2025-04-14T00:00:00,2,NaN
1,Т+: Акции и ДР - безадрес.,1-01-04247-E,RU000A0JPPN4,2,0.1,SUR,NaN,0.06755,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,188307958732,None,None,1.344519e+10,1.261663e+10,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,TQBR,5,NaN,NaN,NaN,NaN,MRKV,РсетВол ао,2025-04-15T00:00:00,2,NaN
2,Т+: Акции и ДР - безадрес.,1-01-04247-E,RU000A0JPPN4,2,0.1,SUR,NaN,0.06700,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,188307958732,None,None,1.344519e+10,1.279553e+10,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,TQBR,5,NaN,NaN,NaN,NaN,MRKV,РсетВол ао,2025-04-16T00:00:00,2,NaN
3,Т+: Акции и ДР - безадрес.,1-01-04247-E,RU000A0JPPN4,2,0.1,SUR,NaN,0.06790,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,188307958732,None,None,1.344519e+10,1.286143e+10,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,TQBR,5,NaN,NaN,NaN,NaN,MRKV,РсетВол ао,2025-04-17T00:00:00,2,NaN
4,Т+: Акции и ДР - безадрес.,1-01-04247-E,RU000A0JPPN4,2,0.1,SUR,NaN,0.06805,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,None,NaN,0.0,0.0,0.0,0,188307958732,None,None,1.344519e+10,1.292734e+10,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,TQBR,5,NaN,NaN,NaN,NaN,MRKV,РсетВол ао,2025-04-18T00:00:00,2,NaN


In [115]:
HistoryStockNdm = RuDataDF("HistoryStockNdm").df
HistoryStockNdm.head()

HistoryStockNdm started
HistoryStockNdm finished. HistoryStockNdm shape (11698, 47)


,boardname,closeval,currencyid,facevalue,isin,issuesize,listname,matdate,numtrades,openval,prev,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Размещение - адрес.,0.0,SUR,500.0,RU000A102H34,10000000,1,2027-12-14T00:00:00,0,0.0,100.0,RU35004STV0,NaN,NaN,NaN,NaN,subfederal_bond,0.0,0,NaN,NaN,NaN,None,NaN,None,6.2,7.73,None,2020-12-15T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSAU,2,NaN,NaN,NaN,NaN,RU000A102H34,СтаврКрай4,2025-03-31T00:00:00,3,NaN
1,Размещение - адрес.,0.0,SUR,500.0,RU000A102H34,10000000,1,2027-12-14T00:00:00,0,0.0,100.0,RU35004STV0,NaN,NaN,NaN,NaN,subfederal_bond,0.0,0,NaN,NaN,NaN,None,NaN,None,6.2,7.73,None,2020-12-15T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSAU,2,NaN,NaN,NaN,NaN,RU000A102H34,СтаврКрай4,2025-04-01T00:00:00,3,NaN
2,Размещение - адрес.,0.0,SUR,500.0,RU000A102H34,10000000,1,2027-12-14T00:00:00,0,0.0,100.0,RU35004STV0,NaN,NaN,NaN,NaN,subfederal_bond,0.0,0,NaN,NaN,NaN,None,NaN,None,6.2,7.73,None,2020-12-15T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSAU,2,NaN,NaN,NaN,NaN,RU000A102H34,СтаврКрай4,2025-04-02T00:00:00,3,NaN
3,Размещение - адрес.,0.0,SUR,500.0,RU000A102H34,10000000,1,2027-12-14T00:00:00,0,0.0,100.0,RU35004STV0,NaN,NaN,NaN,NaN,subfederal_bond,0.0,0,NaN,NaN,NaN,None,NaN,None,6.2,7.73,None,2020-12-15T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSAU,2,NaN,NaN,NaN,NaN,RU000A102H34,СтаврКрай4,2025-04-03T00:00:00,3,NaN
4,Размещение - адрес.,0.0,SUR,500.0,RU000A102H34,10000000,1,2027-12-14T00:00:00,0,0.0,100.0,RU35004STV0,NaN,NaN,NaN,NaN,subfederal_bond,0.0,0,NaN,NaN,NaN,None,NaN,None,6.2,7.73,None,2020-12-15T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSAU,2,NaN,NaN,NaN,NaN,RU000A102H34,СтаврКрай4,2025-04-04T00:00:00,3,NaN


In [116]:
HistoryStockCcp = RuDataDF("HistoryStockCcp").df
HistoryStockCcp.head()

HistoryStockCcp started
HistoryStockCcp finished. HistoryStockCcp shape (39340, 54)


,accint,admittedquote,admittedvalue,bid,boardname,closeperiod,closeval,currencyid,duration,facevalue,highbid,isin,issuesize,legalcloseprice,legalopenprice,listname,lowoffer,marketprice,marketprice2,marketprice3,marketprice3tradesvalue,matdate,mp2valtrd,mpvaltrd,numtrades,offer,openperiod,openval,prev,prevlegalcloseprice,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,3.96,None,None,None,РЕПО с ЦК 7 дн. - безадрес.,None,1.853929e+09,SUR,NaN,1000.0,20.50,RU000A1025A7,409796724,None,None,1,20.20,20.62,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,7,None,None,9.471660e+07,20.25,None,29015RMFS,0.25,None,0.18,NaN,ofz_bond,7.607519e+09,8232672,20.43,None,None,stock,ccp,EQWP,4,20.50,20.20,20.20,20.50,SU29015RMFS3,ОФЗ 29015,2025-04-30T00:00:00,3,NaN
1,3.96,None,None,None,РЕПО с ЦК - адрес.,None,1.141262e+04,SUR,NaN,1000.0,NaN,RU000A1025A7,409796724,None,None,1,NaN,20.62,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,100,None,None,4.978664e+09,NaN,None,29015RMFS,NaN,None,NaN,NaN,ofz_bond,2.987253e+10,32327286,NaN,None,None,stock,ccp,PSRP,4,20.40,NaN,20.40,NaN,SU29015RMFS3,ОФЗ 29015,2025-04-30T00:00:00,3,NaN
2,39.16,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.151240e+07,SUR,NaN,1000.0,21.46,RU000A1025A7,409796724,None,None,1,21.00,21.47,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,7,None,None,9.535806e+08,20.77,None,29015RMFS,0.68,None,0.56,NaN,ofz_bond,5.483551e+09,5923000,21.19,None,None,stock,ccp,EQRP,4,21.45,21.00,21.00,21.45,SU29015RMFS3,ОФЗ 29015,2025-03-31T00:00:00,1,NaN
3,39.16,None,None,None,РЕПО с ЦК - адрес.,None,1.979730e+03,SUR,NaN,1000.0,NaN,RU000A1025A7,409796724,None,None,1,NaN,21.47,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,104,None,None,1.147240e+03,NaN,None,29015RMFS,NaN,None,NaN,NaN,ofz_bond,6.299687e+05,621,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29015RMFS3,ОФЗ 29015,2025-03-31T00:00:00,1,NaN
4,39.75,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,2.876354e+06,SUR,NaN,1000.0,21.80,RU000A1025A7,409796724,None,None,1,21.25,21.06,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,416,None,None,9.553837e+08,21.45,None,29015RMFS,0.05,None,0.30,NaN,ofz_bond,1.911371e+10,20606506,21.49,None,None,stock,ccp,EQRP,4,21.80,21.25,21.25,21.50,SU29015RMFS3,ОФЗ 29015,2025-04-01T00:00:00,1,NaN


##### Шаг 3.1. Tranform

In [117]:
TransformHistoryStockBonds = HistoryStockBonds.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockShares.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdm.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcp.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [118]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [119]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [120]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [121]:
Prices.shape

(1745, 16)

#### Шаг 4. Finish

In [122]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [123]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRate[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [124]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [125]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

In [126]:
FullPrices['export_timestamp'] = dt.now().replace(microsecond=0)
FullPrices['Date'] = last_day_month
FullPrices['Source'] = FullPrices['Source'].fillna(0)

In [127]:
with pd.ExcelWriter(f'./data/Output/prices.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices.to_excel(writer)

In [128]:
# with pd.ExcelWriter(f'./data/Output/prices_full.xlsx', datetime_format='dd.mm.yyyy') as writer: 
#     FullPrices.to_excel(writer)

In [129]:
from functions.send_email import main
main("prices.xlsx")